<a href="https://colab.research.google.com/github/Salome-06/PROGCOM-A/blob/main/GP3_CAJERO_SALOM%C3%89_CUBIDES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import time
import os

#Billetes disponibles en el cajero (Inventario global)
billetes = {
    100000: 10,
    50000: 10,
    20000: 10,
    10000: 10,
    5000: 10,
    2000: 10
}

#Usuarios registrados con sus saldos iniciales (CUENTAS INDIVIDUALES)
usuarios = {
    "1234": {"nombre": "Salomé", "saldo": 10_000_000},  #10 millones COP
    "5678": {"nombre": "Carlos", "saldo": 5_000_000},   #5 millones COP
    "9999": {"nombre": "Miguel", "saldo": 20_000_000}  #20 millones COP
}

#Variable global para el usuario actualmente logueado
usuario_actual_pin = None

#FUNCIONES DE UTILIDAD

def limpiar_pantalla():
    """Limpia la consola."""
    os.system('cls' if os.name == 'nt' else 'clear')

#FUNCIONES DE GESTIÓN DEL CAJERO

def consultar_saldo_cajero():
    """Calcula el saldo total de billetes dentro de la máquina."""
    total = sum(denominacion * cantidad for denominacion, cantidad in billetes.items())
    return total

def retirar_dinero_cajero(monto):
    """Lógica para entregar billetes y actualizar el inventario del cajero."""
    global billetes

    if consultar_saldo_cajero() < monto:
        return False, "El cajero no tiene suficiente efectivo disponible."

    original = monto
    entregado = {}
    monto_restante = monto

    for denominacion in sorted(billetes.keys(), reverse=True):
        cantidad_disponible = billetes[denominacion]
        cantidad_necesaria = min(monto_restante // denominacion, cantidad_disponible)
        if cantidad_necesaria > 0:
            entregado[denominacion] = cantidad_necesaria
            monto_restante -= denominacion * cantidad_necesaria

    if monto_restante == 0:
        #Retiro exitoso: Actualizar inventario
        for denom, cant in entregado.items():
            billetes[denom] -= cant

        mensaje = f"✅ Retiro exitoso de ${original:,} COP.\n🧾 Billetes entregados:\n"
        for denom, cant in entregado.items():
            mensaje += f"   {cant} billete(s) de ${denom:,}\n"

        return True, mensaje
    else:
        return False, "No se puede entregar el monto solicitado con los billetes disponibles (sin cambio exacto)."

#FUNCIONES DE CUENTA Y OPERACIONES

def consultar_saldo_individual():
    """Muestra el saldo disponible del usuario logueado."""
    saldo = usuarios[usuario_actual_pin]["saldo"]
    print(f"💳 Tu saldo actual es: ${saldo:,} COP")

def realizar_donacion_post_transaccion():
    """Pregunta al usuario si desea donar después de una transacción exitosa."""
    global usuarios

    #Pregunta inicial
    respuesta = input("\n🎁 ¿Deseas hacer una donación a una fundación antes de terminar? (s/n): ").lower().strip()

    if respuesta != 's':
        return

    print("\n--- 🎁 DONACIÓN VOLUNTARIA ---")
    print("1. $5.000 (Fundación Techo)")
    print("2. $10.000 (Cruz Roja Colombiana)")
    print("3. Otro Monto")
    print("4. Cancelar")

    opcion = input("Selecciona una opción (1-4): ")
    monto_donacion = 0

    if opcion == "1":
        monto_donacion = 5000
    elif opcion == "2":
        monto_donacion = 10000
    elif opcion == "3":
        try:
            monto_donacion = int(input("Ingresa el monto a donar (Múltiplo de 1.000): "))
            if monto_donacion <= 0 or monto_donacion % 1000 != 0:
                print("❌ Monto inválido. Debe ser positivo y múltiplo de 1.000 COP.")
                return
        except ValueError:
            print("❌ Monto inválido.")
            return
    elif opcion == "4":
        print("Donación cancelada.")
        return
    else:
        print("❌ Opción no válida.")
        return

    saldo_actual = usuarios[usuario_actual_pin]["saldo"]
    if monto_donacion > saldo_actual:
        print(f"❌ Saldo insuficiente para donar ${monto_donacion:,} COP.")
        return

    #Descontar la donación del saldo
    usuarios[usuario_actual_pin]["saldo"] -= monto_donacion

    print(f"\n✅ Donación exitosa de ${monto_donacion:,} COP. ¡Gracias por tu generosidad!")
    consultar_saldo_individual() #Mostrar el saldo restante después de la donación

def realizar_retiro():
    """Controla el flujo de retiro, validando contra el saldo del usuario e integra la donación."""
    global usuarios

    #1. Validación de monto
    try:
        monto = int(input("Ingresa el monto a retirar: "))
    except ValueError:
        print("❌ Monto inválido. Debe ser un número entero.")
        return

    if monto % 1000 != 0:
        print("❌ El monto debe ser múltiplo de 1.000 COP.")
        return

    #2. Validación de saldo del usuario
    saldo_actual = usuarios[usuario_actual_pin]["saldo"]
    if monto > saldo_actual:
        print(f"❌ Saldo insuficiente. Tu saldo es ${saldo_actual:,} COP.")
        return

    #3. Intento de retiro del cajero
    exito, mensaje = retirar_dinero_cajero(monto)

    if exito:
        #4. Retiro exitoso: Actualizar el saldo del usuario
        usuarios[usuario_actual_pin]["saldo"] -= monto
        print(mensaje)
        consultar_saldo_individual() # Mostrar el saldo restante del retiro

        #**INTEGRACIÓN SOLICITADA:** Preguntar sobre la donación inmediatamente después del retiro como es normal en cajeros
        realizar_donacion_post_transaccion()

    else:
        print(f"❌ Error en el cajero: {mensaje}")

#FLUJO PRINCIPAL DEL PROGRAMA

def iniciar_sesion():
    """Controla el inicio de sesión y asigna el usuario actual."""
    global usuario_actual_pin

    print("\n💳 Inserta tu tarjeta (simulado)")
    contraseña = input("🔐 Ingresa tu PIN/Contraseña: ")

    if contraseña in usuarios:
        usuario_actual_pin = contraseña
        nombre = usuarios[contraseña]["nombre"]
        print(f"\n👋 Bienvenido/a, {nombre}!")
        return True
    else:
        print("❌ Contraseña incorrecta. Intenta de nuevo.")
        usuario_actual_pin = None
        return False

def menu_usuario():
    """Menú con las opciones principales del cajero."""
    while True:
        limpiar_pantalla()
        nombre = usuarios[usuario_actual_pin]["nombre"]
        print(f"\n--- 🏦 CAJERO BANCARIO - {nombre.upper()} ---")
        print("1. Consultar Saldo de Mi Cuenta")
        print("2. Retirar Dinero")
        print("3. Consultar Saldo Total del Cajero (Información de Admin)")
        print("4. Finalizar Sesión") #Opción 3 de donación eliminada

        opcion = input("Selecciona una opción (1-4): ")

        if opcion == "1":
            consultar_saldo_individual()
            input("\n[Presiona ENTER para continuar...]")
        elif opcion == "2":
            realizar_retiro()
            input("\n[Presiona ENTER para continuar...]")
        elif opcion == "3":
            saldo_cajero = consultar_saldo_cajero()
            print(f"\n💰 Saldo de efectivo disponible en el cajero: ${saldo_cajero:,} COP")
            input("\n[Presiona ENTER para continuar...]")
        elif opcion == "4":
            print("🔒 Sesión finalizada.")
            break
        else:
            print("❌ Opción no válida. Intenta de nuevo.")
            time.sleep(1)

def cajero():
    """Programa principal."""
    while True:
        if iniciar_sesion():
            menu_usuario()

        #Se verifica si el cajero se queda sin billetes después de una sesión
        if consultar_saldo_cajero() == 0:
            print("\n🚫 Cajero fuera de servicio: no hay billetes disponibles.")
            break

        continuar = input("\n¿Deseas que otra persona use el cajero? (s/n): ").lower()
        if continuar != "s":
            print("\n👋 Gracias por usar el cajero. ¡Hasta pronto!")
            break

#Ejecutar el programa
if __name__ == "__main__":
    cajero()


💳 Inserta tu tarjeta (simulado)
🔐 Ingresa tu PIN/Contraseña: 1234

👋 Bienvenido/a, Salomé!

--- 🏦 CAJERO BANCARIO - SALOMÉ ---
1. Consultar Saldo de Mi Cuenta
2. Retirar Dinero
3. Consultar Saldo Total del Cajero (Información de Admin)
4. Finalizar Sesión
Selecciona una opción (1-4): 2
Ingresa el monto a retirar: 200000
✅ Retiro exitoso de $200,000 COP.
🧾 Billetes entregados:
   2 billete(s) de $100,000

💳 Tu saldo actual es: $9,800,000 COP

🎁 ¿Deseas hacer una donación a una fundación antes de terminar? (s/n): s

--- 🎁 DONACIÓN VOLUNTARIA ---
1. $5.000 (Fundación Techo)
2. $10.000 (Cruz Roja Colombiana)
3. Otro Monto
4. Cancelar
Selecciona una opción (1-4): 2

✅ Donación exitosa de $10,000 COP. ¡Gracias por tu generosidad!
💳 Tu saldo actual es: $9,790,000 COP

[Presiona ENTER para continuar...]

--- 🏦 CAJERO BANCARIO - SALOMÉ ---
1. Consultar Saldo de Mi Cuenta
2. Retirar Dinero
3. Consultar Saldo Total del Cajero (Información de Admin)
4. Finalizar Sesión
Selecciona una opción (1-4)